In [1]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain, RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
import numpy as np

In [2]:
# Load PDFs from directory
loader = PyPDFDirectoryLoader("./data")
docs = loader.load()

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
text_split = text_splitter.split_documents(docs)

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")

# Initialize the vector database
vector_db = FAISS.from_documents(text_split, embedding_model)

C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_13656\3629795972.py:12: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama3.2:3b")


In [3]:
# Embedding using Huggingface
# generating text embeddings
huggingface_embedding = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5", # generating high-quality embeddings for English text
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings':True}
)

In [4]:
np.array(huggingface_embedding.embed_query(text_split[0].page_content))
np.array(huggingface_embedding.embed_query(text_split[0].page_content)).shape

(384,)

In [5]:
# Initialize the vector database
db1 = FAISS.from_documents(text_split[:120], huggingface_embedding)


In [6]:
# Query the vector database
query = "WHAT IS HEALTH INSURANCE COVERAGE?"

search = db1.similarity_search(query)
search[0].page_content

'2 U.S. Census Bureau\nWHAT IS HEALTH INSURANCE COVERAGE?\nThis brief presents state-level estimates of health insurance coverage \nusing data from the American Community Survey (ACS). The  \nU.S. Census Bureau conducts the ACS throughout the year; the \nsurvey asks respondents to report their coverage at the time of \ninterview. The resulting measure of health insurance coverage, \ntherefore, reflects an annual average of current comprehensive \nhealth insurance coverage status.* This uninsured rate measures a \ndifferent concept than the measure based on the Current Population \nSurvey Annual Social and Economic Supplement (CPS ASEC). \nFor reporting purposes, the ACS broadly classifies health insurance \ncoverage as private insurance or public insurance. The ACS defines \nprivate health insurance as a plan provided through an employer \nor a union, coverage purchased directly by an individual from an \ninsurance company or through an exchange (such as healthcare.'

In [7]:
# Initialize the retriever
retriever = db1.as_retriever(search_type="similarity",search_kwargs={"k":3})
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceBgeEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000027C69C1F140>, search_kwargs={'k': 3})

In [15]:
import os
HUGGINGFACE_TOKEN = os.getenv("HUGGINGFACE_TOKEN")

In [16]:
from langchain_community.llms import HuggingFaceHub

# Initialize the HuggingFaceHub model
hf=HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature":0.1,"max_length":500}
)

query="What is the health insurance coverage?"
hf.invoke(query)

ValidationError: 1 validation error for HuggingFaceHub
  Value error, Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. [type=value_error, input_value={'repo_id': 'mistralai/Mi...acehub_api_token': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error

In [ ]:
prompt_template="""
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question:{question}

Helpful Answers:
"""

In [ ]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context","question"]
)


In [ ]:
# Create the RetrievalQA chain
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
# Define the query
query = "DIFFERENCES IN THE UNINSURED RATE BY STATE IN 2022"

In [ ]:
# Call the QA chain with our query.
result = retrievalQA.invoke({"query": query})
print(result['result'])

c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)



Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

comparison of ACS and CPS ASEC measures 
of health insurance coverage, refer to < www.
census.gov/topics/health/health-insurance/
guidance.html >.
9 Respondents may have more than one 
health insurance coverage type at the time 
of interview. As a result, adding the total 
number of people with private coverage and 
the total number with public coverage will 
sum to more than the total number with any 
coverage.• From 2021 to 2022, nine states 
reported increases in private 
coverage, while seven reported 
decreases (Appendix Table B-2). 
DIFFERENCES IN THE 
UNINSURED RATE BY STATE 
IN 2022
In 2022, uninsured rates at the 
time of interview ranged across 
states from a low of 2.4 percent 
in Massachusetts to a high of 16.6 
percent in Texas, compared to the 
national rate of 8.0 percent.10 Ten 
of the 15 states with uninsured 
10 The uninsured rates in the Distr

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain, RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import numpy as np
import pandas as pd
from DataAnalyzer import DataAnalyzer
import os

# Set your Hugging Face API token
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# Load the dataset
file_path = "Regions.csv"
df = pd.read_csv(file_path)

# Create LangChain Document objects
documents = [
    Document(page_content=" | ".join([f"{col}: {str(row[col])}" for col in df.columns]))
    for _, row in df.iterrows()
]

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

# Split the documents into chunks
text_split = text_splitter.split_documents(documents)

# Initialize the embedding model
embedding_model = OllamaEmbeddings(model="llama3.2:3b")

# Initialize the vector database
vector_db = FAISS.from_documents(text_split, embedding_model)

# Embedding using Huggingface
huggingface_embedding = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",  # High-quality embeddings for English text
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# Create the FAISS vector store
db1 = FAISS.from_documents(text_split[:120], huggingface_embedding)

# Initialize the retriever
retriever = db1.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# Initialize the HuggingFaceHub model
hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature": 0.1, "max_length": 500}
)

# Define the prompt template
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

{context}
Question: {question}

Helpful Answers:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# Create the RetrievalQA chain
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# Initialize the DataAnalyzer
analyzer = DataAnalyzer(df, llm=hf)  # Use hf instead of retrievalQA

# Define the query
query = analyzer.analysis_data()

# Call the QA chain with our query
result = retrievalQA.invoke({"query": query})
print(result['result'])

c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)
c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please o


Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context

region_id: 19 | sales_district: Seattle | sales_region: North West

region_id: 23 | sales_district: Salem | sales_region: North West

region_id: 22 | sales_district: Portland | sales_region: North West
Question: 
        You are a data analyst. You are provided with a dataset about <class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   region_id       109 non-null    int64 
 1   sales_district  109 non-null    object
 2   sales_region    109 non-null    object
dtypes: int64(1), object(2)
memory usage: 2.7+ KB

        Here is the dataset structure:
        <class 'pandas.core.frame.DataFrame'>
RangeIndex: 109 entries, 0 to 108
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          

In [ ]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain_community.llms import HuggingFaceHub
import pandas as pd
import os

# تأكد من وجود التوكن الخاص بك على Hugging Face
os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""

# تحميل بيانات CSV
file_path = "Regions.csv"
df = pd.read_csv(file_path)

# إنشاء مستندات من البيانات
documents = [
    Document(page_content=" | ".join([f"{col}: {str(row[col])}" for col in df.columns]))
    for _, row in df.iterrows()
]

# تقسيم النصوص إلى أجزاء صغيرة
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
text_split = text_splitter.split_documents(documents)

# تهيئة نموذج التضمين
embedding_model = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    model_kwargs={'device': 'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

# إنشاء قاعدة بيانات المتجهات
vector_db = FAISS.from_documents(text_split, embedding_model)

# إعداد مسترجع المعلومات
retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})

# إعداد نموذج اللغة من Hugging Face
hf = HuggingFaceHub(
    repo_id="mistralai/Mistral-7B-v0.1",
    model_kwargs={"temperature": 0.1, "max_length": 500}
)

# إعداد قالب التوجيه
prompt_template = """
Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context.

{context}
Question: {question}

Helpful Answers:
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

# إعداد سلسلة استرجاع الأسئلة والإجابات
retrievalQA = RetrievalQA.from_chain_type(
    llm=hf,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

# تحديد الاستعلام لتحليل البيانات
query = "Analyze this dataset based on statistical trends and patterns."

# تنفيذ التحليل
result = retrievalQA.invoke(query)

# طباعة النتيجة
print("Analysis Result:")
print(result.get('result', 'No result found'))

c:\Users\AHMED ABD ELGWAD\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\utils\_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Analysis Result:

Use the following piece of context to answer the question asked.
Please try to provide the answer only based on the context.

region_id: 105 | sales_district: Victoria | sales_region: Canada West

region_id: 42 | sales_district: San Francisco | sales_region: Central West

region_id: 31 | sales_district: San Francisco | sales_region: Central West
Question: Analyze this dataset based on statistical trends and patterns.

Helpful Answers:
- The sales_district "San Francisco" appears twice, both times in the sales_region "Central West".
- The sales_region "Canada West" appears only once, associated with the sales_district "Victoria".
- The sales_region "Central West" appears twice, associated with the sales_district "San Francisco" both times.
- There are no duplicate region_id values in the dataset.
- The dataset contains 3 unique region_id values, 2 unique sales_district values, and 2 unique sales_region values.


In [1]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import pandas as pd
import fitz  # PyMuPDF لقراءة ملفات PDF
import os  # للتحقق من وجود الملف
from DataAnalyzer import DataAnalyzer  # استيراد الكلاس من الملف الآخر

# 1. تحميل ملف القواعد (PDF) من الذاكرة
def load_analysis_rules_from_memory(pdf_content):
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    rules = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        rules += page.get_text()
    return rules

# 2. إنشاء وثائق (Documents) من القواعد
def create_documents_from_rules(rules):
    return [Document(page_content=rules)]

# 3. تدريب نظام RAG على القواعد
def train_rag_system(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)

    embedding_model = OllamaEmbeddings(model="llama2")
    vector_db = FAISS.from_documents(texts, embedding_model)
    
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    llm = Ollama(model="llama2")  # استخدام نموذج Ollama
    
    retrievalQA = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    
    return retrievalQA, llm  # إرجاع retriever + llm لاستخدامه في DataAnalyzer

# 4. تحميل ملف CSV
def load_csv(file_path):
    if not os.path.exists(file_path):
        raise FileNotFoundError(f"File not found: {file_path}")
    return pd.read_csv(file_path)

# الخطوات الرئيسية
if __name__ == "__main__":
    # تحميل ملف القواعد (PDF) وتحليله
    with open("storying.pdf", "rb") as file:
        pdf_content = file.read()
    
    rules = load_analysis_rules_from_memory(pdf_content)
    documents = create_documents_from_rules(rules)
    
    # تدريب نموذج RAG
    retrievalQA, llm = train_rag_system(documents)
    
    # تحميل بيانات CSV
    csv_path = r"D:\My-Githup\Axiora\agent\Regions.csv"
    try:
        df = load_csv(csv_path)
    except FileNotFoundError as e:
        print(e)
        exit(1)
    
    # إنشاء كائن `DataAnalyzer`
    analyzer = DataAnalyzer(df, llm=llm)  # استخدام LLM في التحليل

    # تنفيذ `data_analysis` من DataAnalyzer
    query = analyzer.analysis_data()

    # استخدام RetrievalQA للإجابة على الاستفسار
    result = retrievalQA.invoke({"query": query})
    
    # عرض النتيجة
    print("Final Analysis Result:")
    print(result['result'])


C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_14784\2695014261.py:30: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding_model = OllamaEmbeddings(model="llama2")


ValueError: Error raised by inference API HTTP code: 500, {"error":{}}

In [5]:
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import pandas as pd
import fitz  # PyMuPDF لقراءة ملفات PDF
import os  # للتحقق من وجود الملف
from DataAnalyzer import DataAnalyzer  # استيراد الكلاس من الملف الآخر

# 🔹 مسار تخزين FAISS لتجنب إعادة حساب embeddings
FAISS_DB_PATH = "faiss_index"

# 1️⃣ تحميل ملف القواعد (PDF) من الذاكرة
def load_analysis_rules_from_memory(pdf_content):
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    rules = "\n".join(page.get_text() for page in doc)
    return rules

# 2️⃣ إنشاء وثائق (Documents) من القواعد
def create_documents_from_rules(rules):
    return [Document(page_content=rules)]

# 3️⃣ تدريب أو تحميل نموذج RAG مع FAISS
def train_rag_system(documents):
    """تدريب أو تحميل نموذج RAG مع FAISS لتجنب إعادة حساب embeddings"""
    
    if os.path.exists(FAISS_DB_PATH):
        print("\n🔄 Loading existing FAISS index...")
        vector_db = FAISS.load_local(FAISS_DB_PATH, OllamaEmbeddings(model="llama2"))
    else:
        print("\n🛠️ Generating new embeddings and saving FAISS index...")
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
        texts = text_splitter.split_documents(documents)
        
        embedding_model = OllamaEmbeddings(model="llama2")
        vector_db = FAISS.from_documents(texts, embedding_model)
        
        # 🔥 حفظ قاعدة البيانات للمرات القادمة
        vector_db.save_local(FAISS_DB_PATH)
    
    # 🔹 تحويل FAISS إلى `retriever`
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    
    # 🔹 إنشاء نموذج LLM
    llm = Ollama(model="llama2")
    
    retrievalQA = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    
    return retrievalQA, llm

# 4️⃣ تحميل ملف CSV
def load_csv(file_path):
    return pd.read_csv(file_path)

# 5️⃣ تحليل البيانات باستخدام DataAnalyzer و RAG
def analyze_data_with_rag(analyzer, retrievalQA, df):
    """تحليل البيانات باستخدام DataAnalyzer ثم تحسينها باستخدام RAG"""
    
    query = analyzer.analysis_data()  # تحليل البيانات باستخدام DataAnalyzer
    
    # تحليل البيانات باستخدام RAG
    results = []
    for _, row in df.iterrows():
        data_row = " | ".join([f"{col}: {str(row[col])}" for col in df.columns])
        final_query = f"{query} \n\nAnalyze this data based on the rules: {data_row}"
        
        result = retrievalQA.invoke({"query": final_query})
        results.append(result['result'])
    
    return results

# 🎯 الخطوات الرئيسية
if __name__ == "__main__":
    # ✅ تحميل ملف القواعد (PDF) وتحليله
    with open("storying.pdf", "rb") as file:
        pdf_content = file.read()
    
    rules = load_analysis_rules_from_memory(pdf_content)
    documents = create_documents_from_rules(rules)
    
    # ✅ تدريب نموذج RAG أو تحميل FAISS
    retrievalQA, llm = train_rag_system(documents)
    
    # ✅ تحميل بيانات CSV
    df = load_csv("Regions.csv")  

    # ✅ إنشاء كائن `DataAnalyzer`
    analyzer = DataAnalyzer(df, llm=llm)  

    # ✅ تنفيذ `data_analysis` وتحليل البيانات باستخدام RAG
    analysis_results = analyze_data_with_rag(analyzer, retrievalQA, df)

    # ✅ عرض النتائج
    for i, result in enumerate(analysis_results):
        print(f"\n🔍 Analysis for row {i+1}:")
        print(result)



🛠️ Generating new embeddings and saving FAISS index...


ValueError: Error raised by inference API HTTP code: 500, {"error":"timed out waiting for llama runner to start - progress 0.49 - "}

In [ ]:
## importance
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import Ollama
import pandas as pd
import fitz  # PyMuPDF لقراءة ملفات PDF
import os  # للتحقق من وجود الملف
from DataAnalyzer import DataAnalyzer  # استيراد الكلاس من الملف الآخر

# 1. تحميل ملف القواعد (PDF) من الذاكرة
def load_analysis_rules_from_memory(pdf_content):
    doc = fitz.open(stream=pdf_content, filetype="pdf")
    rules = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        rules += page.get_text()
    return rules

# 2. إنشاء وثائق (Documents) من القواعد
def create_documents_from_rules(rules):
    return [Document(page_content=rules)]

# 3. تدريب نظام RAG على القواعد
def train_rag_system(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    texts = text_splitter.split_documents(documents)

    embedding_model = OllamaEmbeddings(model="llama2")
    vector_db = FAISS.from_documents(texts, embedding_model)
    
    retriever = vector_db.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    llm = Ollama(model="llama2")  # استخدام نموذج Ollama
    
    retrievalQA = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True
    )
    
    return retrievalQA, llm  # إرجاع retriever + llm لاستخدامه في DataAnalyzer

# 4. تحميل ملف CSV
def load_csv(file_path):
    return pd.read_csv(file_path)

# الخطوات الرئيسية
if __name__ == "__main__":
    # تحميل ملف القواعد (PDF) وتحليله
    with open("storying.pdf", "rb") as file:
        pdf_content = file.read()
    
    rules = load_analysis_rules_from_memory(pdf_content)
    documents = create_documents_from_rules(rules)
    
    # تدريب نموذج RAG
    retrievalQA, llm = train_rag_system(documents)
    
    # تحميل بيانات CSV بدون الحاجة لمسار كامل
    df = load_csv("Regions.csv")  

    # إنشاء كائن `DataAnalyzer`
    analyzer = DataAnalyzer(df, llm=llm)  # استخدام LLM في التحليل

    # تنفيذ `data_analysis` من DataAnalyzer
    query = analyzer.analysis_data()

    # استخدام RetrievalQA للإجابة على الاستفسار
    result = retrievalQA.invoke({"query": query})
    
    # عرض النتيجة
    print("Final Analysis Result:")
    print(result['result'])


C:\Users\AHMED ABD ELGWAD\AppData\Local\Temp\ipykernel_14784\2324853806.py:34: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama2")  # استخدام نموذج Ollama
d:\My-Githup\Axiora\agent\huggingface\DataAnalyzer.py:35: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  analysis_chain = LLMChain(llm=self.llm, prompt=analysis_template)
d:\My-Githup\Axiora\agent\huggingface\DataAnalyzer.py:38: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  analysis = analysis_

Final Analysis Result:
Based on the information provided in the question, here are some possible answers to the recommended actions for the organization:

1. Regionalization of Strategies: Develop regional-specific marketing strategies to maximize revenue potential. This could involve tailoring products or services to meet the unique needs of each region, as well as investing in local marketing campaigns and partnerships.
2. Segmentation of Sales Districts: Segment sales districts based on their characteristics (e.g., demographics, psychographics) to better understand customer preferences and behaviors. This could involve creating customer personas or segments and developing targeted marketing strategies for each segment.
3. Investigation of Outliers: Investigate the outliers in the sales region data to identify specific marketing strategies or customer segments that are driving revenue growth. This could involve conducting market research or analyzing customer data to understand their